### DecisionTreeClassifier

In [1]:
import numpy as np
import sys, re
import pandas as pd
import codecs

### Vocabulary dictionary : word_idx_map

In [2]:
def build_data_cv(data_folder, cv=10):
    """
    Loads data and split into 10 folds.
    """
    revs = []
    label_0_file, label_1_file, label_2_file = data_folder #["__label__0.txt","__label__1.txt", "__label__2.txt"]    
    word_idx_map = dict()
    word_idx = 1
    for label, file in enumerate(data_folder): 
        with codecs.open(file, "rb", "utf-8", "ignore") as f:
            for line in f:       
                rev = []
                rev.append(line.strip())
                orig_rev = " ".join(rev).lower()

                words = set(orig_rev.split())
                for word in words:
                    if word not in word_idx_map:
                        word_idx_map[word] = word_idx
                        word_idx += 1

                datum  = {"y": label, 
                          "text": orig_rev,                             
                          "num_words": len(orig_rev.split()),
                          "split": np.random.randint(0, cv)}
                revs.append(datum)
    return revs, word_idx_map

In [3]:
data_folder = ["__label__0.txt","__label__1.txt", "__label__2.txt"]    

print("loading data...")      
revs, word_idx_map = build_data_cv(data_folder, cv=10)
print("Data loaded")

loading data...
Data loaded


### Dataset content
Get data from twitter (reference: http://tech.wonderpla.net/entry/2017/10/10/110000)
- Label 0
    - KEYWORD = "芸能 OR アニメ OR 漫画 OR ドラマ OR ゲーム"            #エンタメ系のキーワード
    - CLASS_LABEL = "\__label__0"

- Label 1
    - KEYWORD = "美容 OR サロン OR エステ OR 化粧 OR 保湿"            #美容系のキーワード
    - CLASS_LABEL = "\__label__1"

- Label 2
    - KEYWORD = "日常 OR 料理 OR 家事 OR 収納 OR 家具"            #暮らし系のキーワード
    - CLASS_LABEL = "\__label__2"

In [4]:
label_names = ['エンタメ系', '美容系', '暮らし系']

#### See data sample

In [5]:
df = pd.DataFrame(revs)

# label 0: Entertainment 
df[df['y'] == 0].sample(2)

,num_words,split,text,y
682,24,6,anthem ていう ゲーム warframe と タイタンフォール を 足し て 2 で ...,0
387,11,5,ナレーター を 無視 する ゲーム です 。 ほんま ツボ # icey,0


In [6]:
# label 1 : Beauty
df[df['y'] == 1].sample(2)

,num_words,split,text,y
790,55,7,月 に 一 度 の 美容 院 は 、 前髪 の カット と 、 カラー と 、 エクステ 頼...,1
1017,68,8,ダイエット 効果 が ある として 流行 の ヨガ や 太極拳 。 これら の 基本 は 腹...,1


In [7]:
# label 2: Life
df[df['y'] == 2].sample(2)

,num_words,split,text,y
1520,31,0,和牛 水田 「 2 人 で 飯 いっ たら 連れ の 料理 が 先 に 出 た 、 17 分...,2
1619,78,9,虐待 さ れ て 死ん だ 子供 に は 同情 できる のに 日常 的 に 虐待 さ れ 続...,2


In [8]:
vocab_size = len(word_idx_map)
print('revs',len(revs)) # number of sentence
print('word_idx_map', vocab_size)

revs 2068
word_idx_map 10329


In [9]:
revs[1]

{'y': 0,
 'text': 'え 、 サラリーマン し ながら 、 商業 漫画 の 仕事 を し て 、 ツイッター に 定期 的 に 漫画 を あげ て 、 さらに コミケ の 原稿 を 作る ！ ？',
 'num_words': 32,
 'split': 2}

### Make BOW vectors
make each sentence an BOW vector using word_idx_map

In [10]:
def make_bow_from_sent(revs, word_idx_map, cv, vocab_size):
    """
    Transforms sentences into a 2-d matrix.
    """
    train, test = [], []
    for rev in revs:
        sent = rev["text"]
        bow_vec = np.zeros(vocab_size + 1)
        bow_vec[0] = rev['y'] # Add true label at the beginning
        words = sent.split()
        for word in words:
            bow_vec[word_idx_map[word]] += 1

        if rev["split"]== cv:  # "split" is random number of np.random.randint(0,10)
            test.append(bow_vec)        
        else:  
            train.append(bow_vec)   
    train = np.array(train, dtype="int")
    test = np.array(test, dtype="int")
    return [train, test] 

In [11]:
cv = 0
datasets = make_bow_from_sent(revs, word_idx_map, cv, vocab_size)

#### Example of sentence to BOW

In [12]:
revs[0]

{'y': 0,
 'text': 'かっこいい 、 わくわく する 漫画 も 気 に なっ てる けど なかなか 読め て ない ん だ よ ねぇ 、 満喫 行こ う か な と 思っ て も 家 で ゴロゴロ っと し ちゃう',
 'num_words': 35,
 'split': 4}

In [13]:
datasets[0][0]

array([0, 1, 1, ..., 0, 0, 0])

### Implement DecisionTreeClassifier

In [14]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state = 0)

In [15]:
train_image = datasets[0][:, 1:]
train_label = datasets[0][:, 0]
test_image = datasets[1][:, 1:]
test_label = datasets[1][:, 0]

In [16]:
print('train data size:', train_image.shape)
print('train data size:', train_label.shape)
print('test data size:', test_image.shape)
print('train data size:', test_label.shape)

train data size: (1874, 10329)
train data size: (1874,)
test data size: (194, 10329)
train data size: (194,)


In [17]:
clf.fit(train_image, train_label)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [18]:
print('Accuracy:', clf.score(test_image, test_label))

Accuracy: 0.994845360825


In [19]:
print('Accuracy:', clf.score(train_image, train_label))

Accuracy: 0.996798292423


### Predict single inputted text

#### Input the text to predict (change the text below)

In [20]:
# text_to_predict = "お風呂掃除でいつも落ちなかった溝にある黒カビが家事えもんの塩素系漂白剤＋片栗粉でほぼ真っ白になって感動 家事えもんのテクニック凄い～！"

text_to_predict = "コスメの最安値が見つけられるアプリ💄💋メイク動画とか 美容情報も載ってるし最高😆🙌📲http://goo.gl/K5Fmea 女子にはほんとに助かる〜💗"

In [25]:
from get_data_from_twitter import process_single_txt

In [22]:
def make_bow_from_single_sent(sent, word_idx_map, vocab_size):
    """
    Transforms sentences into a 2-d matrix.
    """
    test = []
    bow_vec = np.zeros(vocab_size + 1)
    bow_vec[0] = 0 # Add dummy true label at the beginning
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            bow_vec[word_idx_map[word]] += 1

    test.append(bow_vec)
    test = np.array(test, dtype="int")
    return test


In [23]:
processed_sent = process_single_txt(text_to_predict)
single_bow = make_bow_from_single_sent(processed_sent, word_idx_map, vocab_size)

1行処理済み


In [24]:
predected = clf.predict(single_bow[:, 1:]).item()
print('Predicted:', predected, label_names[predected])

Predicted: 1 美容系
